<a href="https://colab.research.google.com/github/kategavrishina/info-search/blob/master/HW4/hw4_semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gensim --upgrade

In [ ]:
! pip install natasha

Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  

In [3]:
%load_ext autoreload

from gensim.models import KeyedVectors

In [4]:
import urllib

In [6]:
urllib.request.urlretrieve("https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz", "araneum_none_fasttextcbow_300_5_2018.tgz")

('araneum_none_fasttextcbow_300_5_2018.tgz',
 <http.client.HTTPMessage at 0x7f7ba391c8d0>)

In [7]:
! tar xvfz araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [57]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

In [58]:
! pip install pymorphy2

In [59]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [60]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

In [62]:
import re

In [63]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()

morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [64]:
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [65]:
def preprocess(text):
    reg = re.compile('[^а-яА-Я ]')
    text = reg.sub('', text.lower().strip())
    words = text.strip().split()
    s = []
    for word in words:
        if word != '':
            word = morph.parse(word)[0].normal_form
            if word not in russian_stopwords:
                s.append(word)
    return s

In [66]:
def preprocess_with_natasha(text: str) -> str:
    extractors = [
              dates_extractor,
              money_extractor,
              addr_extractor
    ]
    
    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    indices = []
    for span in doc.spans:
        if span.type == "PER":
            indices.append((span.start, span.stop))
    for pair in indices[::-1]:
        text = text[:pair[0]] + text[pair[1]:]

    for extractor in extractors:
        idx = []
        for match in extractor(text):
            idx.append((match.start, match.stop))
        for pair in idx[::-1]:
            text = text[:pair[0]] + text[pair[1]:]
    
    return preprocess(text)

In [67]:
import numpy as np

In [68]:
import pandas as pd

In [69]:
ans = pd.read_excel('answers_base.xlsx')

In [70]:
answers = pd.DataFrame(ans[['Номер связки', 'Текст ответа']])

In [71]:
ans['Вопросы'] = ans['Текст вопросов'].apply(lambda text: text.split('\n'))

In [72]:
ans = ans.explode('Вопросы')

In [73]:
ans.reset_index(drop=True, inplace=True)
ans.drop(['Текст вопросов', 'Текст ответа', 'Тематика'], axis=1, inplace=True)

In [74]:
ans.head()

,Номер связки,Вопросы
0,57,У ребенка в школе продлили каникулы. Могу ли я...
1,57,Больничный лист?
2,57,"Есть ли компенсация, в случае если есть разниц..."
3,57,как оплачивается больничный при коронавирусе?
4,57,"Я контактный, дадут ли больничный?"


In [75]:
q = pd.read_excel('queries_base.xlsx', usecols='A, B', names=['Вопросы', 'Номер связки'])

In [76]:
q = q[['Номер связки', 'Вопросы']]

In [77]:
data = pd.concat([ans, q], ignore_index=True)

In [78]:
from tqdm.auto import tqdm

tqdm.pandas()

In [79]:
data['Preprocessed'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(str(text))))

In [80]:
data['Natasha'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess_with_natasha(str(text))))

In [81]:
data.head()

,Номер связки,Вопросы,Preprocessed,Natasha
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бол...,ребёнок школа продлить каникулы мочь взять бол...
1,57.0,Больничный лист?,больничный лист,больничный лист
2,57.0,"Есть ли компенсация, в случае если есть разниц...",компенсация случай разница оплата больничный з...,компенсация случай разница оплата больничный з...
3,57.0,как оплачивается больничный при коронавирусе?,оплачиваться больничный коронавирус,оплачиваться больничный коронавирус
4,57.0,"Я контактный, дадут ли больничный?",контактный дать больничный,контактный дать больничный


In [82]:
data_train, data_test = data[:2150], data[2150:]

In [83]:
def get_vector_mean(lemmas):
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        try:
            lemmas_vectors[idx] = model[lemma]
        except AttributeError:
            continue

    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)
    
    return vec

In [84]:
matrix = np.zeros((len(data), model.vector_size))
for idx, text in enumerate(data.Preprocessed):
    matrix[idx] = get_vector_mean(text.split())

matrix_natasha = np.zeros((len(data), model.vector_size))
for idx, text in enumerate(data.Natasha):
    matrix_natasha[idx] = get_vector_mean(text.split())

In [85]:
matrix_train, matrix_test = matrix[:2150], matrix[2150:]
matrix_train_n, matrix_test_n = matrix_natasha[:2150], matrix_natasha[2150:]

In [86]:
def get_classic_answer(vec):
    res = vec.dot(matrix_train.T)
    return int(data_train.loc[res.argmax()]['Номер связки'])

In [87]:
data_test['classic'] = [get_classic_answer(i) for i in matrix_test]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
data_test['classic_natasha'] = [get_classic_answer(i) for i in matrix_test_n]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
data_test[data_test['Номер связки'] == data_test.classic].shape[0]/data_test.shape[0]

0.11935483870967742

In [90]:
data_test[data_test['Номер связки'] == data_test.classic_natasha].shape[0]/data_test.shape[0]

0.11720430107526882

In [91]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

In [92]:
def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        try:
            lemmas_vectors[idx] = normalize_vec(model[lemma])
        except AttributeError:
            continue
            
    return lemmas_vectors  

In [93]:
list_of_mtx = []
for idx, text in enumerate(data.Preprocessed):
    list_of_mtx.append(create_doc_matrix(text))

list_of_mtx_n = []
for idx, text in enumerate(data.Natasha):
    list_of_mtx_n.append(create_doc_matrix(text))

In [94]:
list_train, list_test = list_of_mtx[:2150], list_of_mtx[2150:]
list_train_n, list_test_n = list_of_mtx_n[:2150], list_of_mtx_n[2150:]

In [95]:
def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        try:
            sim = reduce_func(sim, axis=axis)
        except:
            continue
        sims.append(sim.sum())
    return np.argmax(sims)

In [96]:
data_test['experiment'] = [data_train.loc[search(list_train, mat)]['Номер связки'] for mat in tqdm(list_test)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
data_test['experiment_natasha'] = [data_train.loc[search(list_train_n, mat)]['Номер связки'] for mat in tqdm(list_test_n)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
data_test[data_test['Номер связки'] == data_test.experiment].shape[0]/data_test.shape[0]

0.10215053763440861

In [99]:
data_test[data_test['Номер связки'] == data_test.experiment_natasha].shape[0]/data_test.shape[0]

0.3473118279569892

In [100]:
data_test.head()

,Номер связки,Вопросы,Preprocessed,Natasha,classic,classic_natasha,experiment,experiment_natasha
2150,6.0,Здравствуйте. У нас планово должны были положи...,здравствуйте планово должный положить стациона...,здравствуйте планово должный положить стациона...,132,327,37.0,308.0
2151,70.0,Добрый день! Проинформируйте пожалуйста по дан...,добрый день проинформировать пожалуйста дать в...,добрый день проинформировать пожалуйста дать в...,79,79,132.0,79.0
2152,308.0,Здравствуйте. Подскажите пожалуйста. Вчера при...,здравствуйте подсказать пожалуйста вчера приле...,здравствуйте подсказать пожалуйста вчера приле...,45,327,37.0,308.0
2153,6.0,Добрый день! Сын сдал анализ на коронавирус 19...,добрый день сын сдать анализ коронавирус авгус...,добрый день сын сдать анализ коронавирус сегод...,45,45,37.0,308.0
2154,308.0,"Здравствуйте, скажите пожалуйста, после однодн...",здравствуйте сказать пожалуйста однодневный гр...,здравствуйте сказать пожалуйста однодневный гр...,327,327,132.0,308.0
